Python version should be `3.11`.

Scikit-learn version should be `1.5.2`.

In [72]:
import sklearn
import platform

print(f"Python version is {platform.python_version()}")
print(f"Scikit-learn version is {sklearn.__version__}")

Python version is 3.11.9
Scikit-learn version is 1.5.2


## Import data

In [73]:
import pandas as pd
df1 = pd.read_parquet("validated_data/cleaned1.parquet")
df2 = pd.read_parquet("validated_data/cleaned2.parquet")
df3 = pd.read_parquet("validated_data/cleaned3.parquet")

dfs = [df1, df2, df3]
for i, df in enumerate(dfs):
    dfs[i] = df[df["Faulty"] == False]
    print(f"Building {i+1} has {len(dfs[i])} out of {len(df)} valid rows.")
df1, df2, df3 = dfs

Building 1 has 376821 out of 2097150 valid rows.
Building 2 has 22716 out of 520938 valid rows.
Building 3 has 467517 out of 1942515 valid rows.


## Data preparation

In [3]:
#df2.groupby("Zone_name").count()

Only keep columns with useful information.
- `Slab_temp` and `Dew_temp` values do not vary in building 1
- `building_no` is the same value within dataframes
- `Fan_on_group` and `Cumulative_fan_on_mins` values were calculated using the target variable `Fan_status`
- `Date` and `Time` cannot be easily converted to numeric values
- `Year` values should probably not be used for prediction
- `Damper_open_group` and `Louver_open_group` are categorical labels
- Cumulative metrics probably won't be available when trying to predict future values
- `Louver_status` values are missing in building 3

In [74]:
exclude = ["building_no", "Fan_on_group", "Date", "Time", "Year", "Damper_open_group", "Louver_open_group", "Faulty", "Cumulative_fan_on_mins", "Cumulative_damper_open_mins", "Cumulative_louver_open_mins"]
exclude_df1 = exclude + ["Slab_temp", "Dew_temp", "Slab_temp_diff", "Dew_temp_diff"]
exclude_df3 = exclude + ["Louver_status"]

df1 = df1[df1.columns.difference(exclude_df1)]
df2 = df2[df2.columns.difference(exclude)]
df3 = df3[df3.columns.difference(exclude_df3)]
dfs = [df1, df2, df3]

Use `Datetime` to get day of year and minutes past midnight

In [75]:
for df in dfs:
    df["Day_of_Year"] = df["Datetime"].dt.day_of_year
    df["Minutes_past_midnight"] = ((df["Datetime"] - df["Datetime"].dt.normalize()) / pd.Timedelta(minutes=1))
    #df["DOW"] = df["Datetime"].dt.day_name().astype("category")
    del df["Datetime"]

#import seaborn as sns
#sns.heatmap(df1[df1.columns.difference(["Fan_status", "Zone_name", "DOW"])].corr(), cmap="BrBG", vmin=-1, vmax=1)

Remove rows with `NA` values for columns that do not have many `NA` values.

In [76]:
for df in dfs:
    # limit to columns with less than 5% NA values
    cutoff = len(df) // 20

    sums = df.isna().sum()
    sums = sums[sums != 0]
    #print(cutoff)
    #print(sums)
    sums = sums[sums <= cutoff]
    df.dropna(subset=sums.index, inplace=True)

18841
Ambient_temp           3110
Ambient_temp_diff      6074
Damper_status        363206
dtype: int64
1135
Ambient_temp_diff       1
Zone_c02             1762
dtype: int64
23375
Ambient_temp_diff     39
Datetime_diff_mins    13
Dew_temp_diff         48
Slab_temp_diff        38
Zone_temp_diff        41
dtype: int64


We need to transform `Zone_name` from categorical to numeric. We'll use scikit-learn preprocessing for encoding.

In [7]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import TargetEncoder

def encode(categorical_columns: list[int] | list[int]):
    return make_column_transformer((TargetEncoder(random_state=42), categorical_columns), remainder="passthrough")

#categorical_columns = ["Zone_name"]
#categorical_columns = ["Zone_name", "DOW"]
#df1 = pd.get_dummies(df1, columns=categorical_columns)
#dfs[0] = df1
#df2 = pd.get_dummies(df2, columns=categorical_columns)
#dfs[1] = df2
#df3 = pd.get_dummies(df3, columns=categorical_columns)
#dfs[2] = df3

Dump the processed datasets.

In [8]:
from pathlib import Path

save_folder = Path("output/data")
save_folder.mkdir(parents=True, exist_ok=True)
for i, df in enumerate(dfs):
    df.to_parquet(save_folder / f"building{i+1}.parquet")

## Split datasets into training and testing subsets
We use a 80% training, 10% validation, 10% testing split.

In [9]:
from sklearn.model_selection import train_test_split

train1, test1 = train_test_split(df1, test_size=0.2, random_state=42)
val1, test1 = train_test_split(test1, test_size=0.5, random_state=42)
train2, test2 = train_test_split(df2, test_size=0.2, random_state=42)
val2, test2 = train_test_split(test2, test_size=0.5, random_state=42)
train3, test3 = train_test_split(df3, test_size=0.2, random_state=42)
val3, test3 = train_test_split(test3, test_size=0.5, random_state=42)

Use random undersampling to handle unbalanced training sets.

In [10]:
target = "Fan_status"

for train in [train1, train2, train3]:
    counts = train[target].value_counts()
    if counts["On"] > counts["Off"] * 3:
        # data is already in random order, so we can just remove the relevant records after the cutoff
        drop = train[train[target] == "On"].index[counts["Off"] * 3:]
        train.drop(drop, inplace=True)
    elif counts["Off"] > counts["On"] * 3:
        drop = train[train[target] == "Off"].index[counts["On"] * 3:]
        train.drop(drop, inplace=True)

In [11]:
features1 = df1.columns.to_list()
features1.remove(target)
y_train1 = train1[target]
y_val1 = val1[target]
y_test1 = test1[target]
x_train1 = train1[features1]
x_val1 = val1[features1]
x_test1 = test1[features1]

features2 = df2.columns.to_list()
features2.remove(target)
y_train2 = train2[target]
y_val2 = val2[target]
y_test2 = test2[target]
x_train2 = train2[features2]
x_val2 = val2[features2]
x_test2 = test2[features2]

features3 = df3.columns.to_list()
features3.remove(target)
y_train3 = train3[target]
y_val3 = val3[target]
y_test3 = test3[target]
x_train3 = train3[features3]
x_val3 = val3[features3]
x_test3 = test3[features3]

## Saving and loading models

In [12]:
from sklearn.base import BaseEstimator
from joblib import dump, load

def save_model(model: BaseEstimator, filename: str):
    models_folder = Path("output/models")
    models_folder.mkdir(parents=True, exist_ok=True)
    filename = filename + ".joblib"
    dump(model, models_folder / filename)

def load_model(filename: str) -> BaseEstimator | None:
    models_folder = Path("output/models")
    filename = filename + ".joblib"
    filename: Path = models_folder / filename
    if filename.is_file():
        #print("File found!")
        return load(filename)
    #print("File does not exist!")

## Evaluating Model Performance

In [13]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef, confusion_matrix

def calculate_metrics(model: BaseEstimator, X: pd.Series, y: pd.Series, print_report = True):
    y_predict = model.predict(X)
    # SVM predictions take a long time to compute and are unreliable (https://stackoverflow.com/questions/15111408/how-does-sklearn-svm-svcs-function-predict-proba-work-internally)
    #y_proba = model.predict_proba(X)[:,1]

    if print_report:
        print(classification_report(y, y_predict, digits=4))
    
    tn, fp, fn, tp = confusion_matrix(y, y_predict).ravel()

    return {
        "Accuracy": accuracy_score(y, y_predict),
        "f1": f1_score(y, y_predict, pos_label="On"),
        #"ROC AUC": roc_auc_score(y, y_proba),
        #"PR AUC": average_precision_score(y, y_proba, pos_label="On"),
        "Matthews": matthews_corrcoef(y, y_predict),
        "TN": tn,
        "FP": fp,
        "FN": fn,
        "TP": tp,
    }

model_metrics = {f"Building {n+1}": {} for n in range(3)}
tuned_metrics = {f"Building {n+1}": {} for n in range(3)}

## Decision Tree

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

tree = DecisionTreeClassifier(random_state=42)
tree = make_pipeline(encode(["Zone_name"]), tree)
model_name = "Decision Tree"

### Building 1

In [ ]:
tree.fit(x_train1, y_train1)
model_metrics["Building 1"][model_name] = calculate_metrics(tree, x_val1, y_val1)
tuned_metrics["Building 1"][model_name + " (default)"] = calculate_metrics(tree, x_test1, y_test1, print_report=False)
save_model(tree, "decisiontree_building1")

### Building 2

In [ ]:
tree.fit(x_train2, y_train2)
model_metrics["Building 2"][model_name] = calculate_metrics(tree, x_val2, y_val2)
tuned_metrics["Building 2"][model_name + " (default)"] = calculate_metrics(tree, x_test2, y_test2, print_report=False)
save_model(tree, "decisiontree_building2")

### Building 3

In [ ]:
tree.fit(x_train3, y_train3)
model_metrics["Building 3"][model_name] = calculate_metrics(tree, x_val3, y_val3)
tuned_metrics["Building 3"][model_name + " (default)"] = calculate_metrics(tree, x_test3, y_test3, print_report=False)
save_model(tree, "decisiontree_building3")

## Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns

model_name = "Random Forest"
forest = RandomForestClassifier(random_state=42, n_jobs=-1)
forest = make_pipeline(encode(["Zone_name"]), forest)

### Building 1

In [ ]:
forest.fit(x_train1, y_train1)
feature_names = forest[0].get_feature_names_out()
feature_names = [feature.split("__")[1] for feature in feature_names]
sns.barplot(pd.DataFrame(forest[-1].feature_importances_, index=feature_names).T, orient="h")

In [ ]:
model_metrics["Building 1"][model_name] = calculate_metrics(forest, x_val1, y_val1)
tuned_metrics["Building 1"][model_name + " (default)"] = calculate_metrics(forest, x_test1, y_test1, print_report=False)
save_model(forest, "randomforest_building1")

### Building 2

In [ ]:
forest.fit(x_train2, y_train2)
feature_names = forest[0].get_feature_names_out()
feature_names = [feature.split("__")[1] for feature in feature_names]
sns.barplot(pd.DataFrame(forest[-1].feature_importances_, index=feature_names).T, orient="h")

In [ ]:
model_metrics["Building 2"][model_name] = calculate_metrics(forest, x_val2, y_val2)
tuned_metrics["Building 2"][model_name + " (default)"] = calculate_metrics(forest, x_test2, y_test2, print_report=False)
save_model(forest, "randomforest_building2")

### Building 3

In [ ]:
forest.fit(x_train3, y_train3)
feature_names = forest[0].get_feature_names_out()
feature_names = [feature.split("__")[1] for feature in feature_names]
sns.barplot(pd.DataFrame(forest[-1].feature_importances_, index=feature_names).T, orient="h")

In [ ]:
model_metrics["Building 3"][model_name] = calculate_metrics(forest, x_val3, y_val3)
tuned_metrics["Building 3"][model_name + " (default)"] = calculate_metrics(forest, x_test3, y_test3, print_report=False)
save_model(forest, "randomforest_building3")

## Histogram-based Gradient Boosting

In [25]:
from sklearn.ensemble import HistGradientBoostingClassifier

model_name = "Hist Gradient Boosting"
histgb = HistGradientBoostingClassifier(random_state=42, categorical_features=["Zone_name"])

In [ ]:
histgb.fit(x_train1, y_train1)
model_metrics["Building 1"][model_name] = calculate_metrics(histgb, x_val1, y_val1)
tuned_metrics["Building 1"][model_name + " (default)"] = calculate_metrics(histgb, x_test1, y_test1, print_report=False)
save_model(histgb, "histgb_building1")

In [ ]:
histgb.fit(x_train2, y_train2)
model_metrics["Building 2"][model_name] = calculate_metrics(histgb, x_val2, y_val2)
tuned_metrics["Building 2"][model_name + " (default)"] = calculate_metrics(histgb, x_test2, y_test2, print_report=False)
save_model(histgb, "histgb_building2")

In [ ]:
histgb.fit(x_train3, y_train3)
model_metrics["Building 3"][model_name] = calculate_metrics(histgb, x_val3, y_val3)
tuned_metrics["Building 3"][model_name + " (default)"] = calculate_metrics(histgb, x_test3, y_test3, print_report=False)
save_model(histgb, "histgb_building3")

## Logistic Regression

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

model_name = "Logistic Regression"
logit = LogisticRegression(random_state=42)
logit = make_pipeline(encode(["Zone_name"]), SimpleImputer(), StandardScaler(), logit)

In [ ]:
logit.fit(x_train1, y_train1)
model_metrics["Building 1"][model_name] = calculate_metrics(logit, x_val1, y_val1)
tuned_metrics["Building 1"][model_name + " (default)"] = calculate_metrics(logit, x_test1, y_test1, print_report=False)
save_model(logit, "logit_building1")

In [ ]:
logit.fit(x_train2, y_train2)
model_metrics["Building 2"][model_name] = calculate_metrics(logit, x_val2, y_val2)
tuned_metrics["Building 2"][model_name + " (default)"] = calculate_metrics(logit, x_test2, y_test2, print_report=False)
save_model(logit, "logit_building2")

In [ ]:
logit.fit(x_train3, y_train3)
model_metrics["Building 3"][model_name] = calculate_metrics(logit, x_val3, y_val3)
tuned_metrics["Building 3"][model_name + " (default)"] = calculate_metrics(logit, x_test3, y_test3, print_report=False)
save_model(logit, "logit_building3")

## Linear Discriminant Analysis

In [33]:
from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model_name = "Linear Discriminant Analysis"
lda = LinearDiscriminantAnalysis()
lda = make_pipeline(encode(["Zone_name"]), SimpleImputer(), lda)

In [ ]:
lda.fit(x_train1, y_train1)
model_metrics["Building 1"][model_name] = calculate_metrics(lda, x_val1, y_val1)
tuned_metrics["Building 1"][model_name + " (default)"] = calculate_metrics(lda, x_test1, y_test1, print_report=False)
save_model(lda, "lda_building1")

In [ ]:
lda.fit(x_train2, y_train2)
model_metrics["Building 2"][model_name] = calculate_metrics(lda, x_val2, y_val2)
tuned_metrics["Building 2"][model_name + " (default)"] = calculate_metrics(lda, x_test2, y_test2, print_report=False)
save_model(lda, "lda_building2")

In [ ]:
lda.fit(x_train3, y_train3)
model_metrics["Building 3"][model_name] = calculate_metrics(lda, x_val3, y_val3)
tuned_metrics["Building 3"][model_name + " (default)"] = calculate_metrics(lda, x_test3, y_test3, print_report=False)
save_model(lda, "lda_building3")

## Linear Support Vector Machine (SVM)

In [37]:
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVC

model_name = "Linear SVM"
svm = LinearSVC(random_state=42)
svm = make_pipeline(encode(["Zone_name"]), SimpleImputer(), MinMaxScaler(), Normalizer(), svm)

In [ ]:
svm.fit(x_train1, y_train1)
model_metrics["Building 1"][model_name] = calculate_metrics(svm, x_val1, y_val1)
tuned_metrics["Building 1"][model_name + " (default)"] = calculate_metrics(svm, x_test1, y_test1, print_report=False)
save_model(svm, "svm_linear_building1")

In [ ]:
svm.fit(x_train2, y_train2)
model_metrics["Building 2"][model_name] = calculate_metrics(svm, x_val2, y_val2)
tuned_metrics["Building 2"][model_name + " (default)"] = calculate_metrics(svm, x_test2, y_test2, print_report=False)
save_model(svm, "svm_linear_building2")

In [ ]:
svm.fit(x_train3, y_train3)
model_metrics["Building 3"][model_name] = calculate_metrics(svm, x_val3, y_val3)
tuned_metrics["Building 3"][model_name + " (default)"] = calculate_metrics(svm, x_test3, y_test3, print_report=False)
save_model(svm, "svm_linear_building3")

## SVM with RBF Kernel

SVM takes a long time for large datasets. Even after loading the saved model file, making predictions on the buildings took around 1 hour.

In [41]:
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC

model_name = "SVM"
svm = SVC(random_state=42)
svm = make_pipeline(encode(["Zone_name"]), SimpleImputer(), MinMaxScaler(), Normalizer(), svm)

In [42]:
#from sklearn.preprocessing import MinMaxScaler, Normalizer
#from sklearn.impute import SimpleImputer
#from sklearn.svm import LinearSVC
#from sklearn.kernel_approximation import Nystroem

#model_name = "SVM (RBF)"
#svm = LinearSVC(random_state=42)
#svm = make_pipeline(encode(["Zone_name"]), SimpleImputer(), MinMaxScaler(), Normalizer(), Nystroem(), svm)

In [43]:
# Building 1 SVM training took around 3 hours for me. We'll try loading the saved model file instead.
model_file = "svm_rbf_building1"
#if load_model(model_file):
#    print("Loading previously saved model")
#    svm = load_model(model_file)
#    model_metrics["Building 1"][model_name] = calculate_metrics(svm, x_val1, y_val1)
#    tuned_metrics["Building 1"][model_name + " (default)"] = calculate_metrics(svm, x_test1, y_test1, print_report=False)
#else:
#    print("Training model...")
#    svm.fit(x_train1, y_train1)
#    model_metrics["Building 1"][model_name] = calculate_metrics(svm, x_val1, y_val1)
#    tuned_metrics["Building 1"][model_name + " (default)"] = calculate_metrics(svm, x_test1, y_test1, print_report=False)
#    save_model(svm, model_file)

In [ ]:
svm.fit(x_train2, y_train2)
model_metrics["Building 2"][model_name] = calculate_metrics(svm, x_val2, y_val2)
tuned_metrics["Building 2"][model_name + " (default)"] = calculate_metrics(svm, x_test2, y_test2, print_report=False)
save_model(svm, "svm_rbf_building2")

In [45]:
# Building 3 SVM training took around 1 hour for me. We'll try loading the saved model file instead.
model_file = "svm_rbf_building3"
#if load_model(model_file):
#    print("Loading previously saved model")
#    svm = load_model(model_file)
#    model_metrics["Building 3"][model_name] = calculate_metrics(svm, x_val3, y_val3)
#    tuned_metrics["Building 3"][model_name + " (default)"] = calculate_metrics(svm, x_test3, y_test3, print_report=False)
#else:
#    print("Training model...")
#    svm.fit(x_train1, y_train1)
#    model_metrics["Building 3"][model_name] = calculate_metrics(svm, x_val3, y_val3)
#    tuned_metrics["Building 3"][model_name + " (default)"] = calculate_metrics(svm, x_test3, y_test3, print_report=False)
#    save_model(svm, model_file)

## Model Performance
### Building 1

In [ ]:
pd.DataFrame(model_metrics["Building 1"]).T.sort_values("f1", ascending=False)

### Building 2

In [ ]:
pd.DataFrame(model_metrics["Building 2"]).T.sort_values("f1", ascending=False)

### Building 3

In [ ]:
pd.DataFrame(model_metrics["Building 3"]).T.sort_values("f1", ascending=False)

## Model Tuning

Creating and evaluating all model variants takes a long time to complete from scratch. Data will be cached so that future runs will not take as long.

In [49]:
from mlmodel import SCORES
scores = list(SCORES)
save_folder = None

### Decision Tree

In [50]:
from mlmodel import GridSearchDecisionTree
model_name = "Decision Tree"
#save_folder = "output/decision_tree/analysis"
save_folder = None

#### Building 1

In [ ]:
opt1 = GridSearchDecisionTree("building1", n_fits=-1)
opt1.fit(x_train1, y_train1, x_val1, y_val1)
if save_folder:
    print(opt1.save_results(save_folder))

df = opt1.results
df = pd.DataFrame(df[scores], index=df["nodes"])
plt = sns.lineplot(df)
plt.set_xlim(0, len(df))
#plt.set_ylim(0.998, 1.0)
display(plt)

In [ ]:
tuned_metrics["Building 1"][model_name] = calculate_metrics(opt1.model, x_test1, y_test1)
save_model(opt1.model, "decisiontree_building1_tuned")

#### Building 2

In [ ]:
opt2 = GridSearchDecisionTree("building2", n_fits=-1)
opt2.fit(x_train2, y_train2, x_val2, y_val2)
if save_folder:
    print(opt2.save_results(save_folder))

df = opt2.results
df = pd.DataFrame(df[scores], index=df["nodes"])
plt = sns.lineplot(df)
plt.set_xlim(0, len(df))
#plt.set_ylim(0.6, 1.0)
display(plt)

In [ ]:
tuned_metrics["Building 2"][model_name] = calculate_metrics(opt2.model, x_test2, y_test2)
save_model(opt2.model, "decisiontree_building2_tuned")

#### Building 3

In [ ]:
opt3 = GridSearchDecisionTree("building3", n_fits=-1)
opt3.fit(x_train3, y_train3, x_val3, y_val3)
if save_folder:
    print(opt3.save_results(save_folder))

df = opt3.results
df = pd.DataFrame(df[scores], index=df["nodes"])
plt = sns.lineplot(df)
plt.set_xlim(0, len(df))
#plt.set_ylim(0.92, 1.0)
display(plt)

In [ ]:
tuned_metrics["Building 3"][model_name] = calculate_metrics(opt3.model, x_test3, y_test3)
save_model(opt3.model, "decisiontree_building3_tuned")

### Random Forest

In [70]:
from mlmodel import GridSearchRandomForest
model_name = "Random Forest"
#save_folder = "output/random_forest/analysis"
save_folder = None

#### Building 1

In [ ]:
opt1 = GridSearchRandomForest("building1", n_fits=-1)
opt1.fit(x_train1, y_train1, x_val1, y_val1)
if save_folder:
    print(opt1.save_results(save_folder))

opt1.model[-1]

In [ ]:
df = opt1.results
df = pd.DataFrame(df[scores], index=df["min_samples_split"])
plt = sns.lineplot(df)
#plt.set_xlim(0, df.index.max())
#plt.set_ylim(0.98, 1.0)
display(plt)

In [ ]:
tuned_metrics["Building 1"][model_name] = calculate_metrics(opt1.model, x_test1, y_test1)
save_model(opt1.model, "randomforest_building1_tuned")

#### Building 2

In [ ]:
opt2 = GridSearchRandomForest("building2", n_fits=-1)
opt2.fit(x_train2, y_train2, x_val2, y_val2)
if save_folder:
    print(opt2.save_results(save_folder))

opt2.model[-1]

In [ ]:
df = opt2.results
df = pd.DataFrame(df[scores], index=df["min_samples_split"])
plt = sns.lineplot(df)
display(plt)

In [ ]:
tuned_metrics["Building 2"][model_name] = calculate_metrics(opt2.model, x_test2, y_test2)
save_model(opt2.model, "randomforest_building2_tuned")

In [ ]:
opt3 = GridSearchRandomForest("building3", n_fits=-1)
opt3.fit(x_train3, y_train3, x_val3, y_val3)
if save_folder:
    print(opt3.save_results(save_folder))

opt3.model[-1]

In [ ]:
df = opt3.results
df = pd.DataFrame(df[scores], index=df["min_samples_split"])
plt = sns.lineplot(df)
display(plt)

In [ ]:
tuned_metrics["Building 3"][model_name] = calculate_metrics(opt3.model, x_test3, y_test3)
save_model(opt3.model, "randomforest_building3_tuned")

## Tuned Model Performance
### Building 1

In [ ]:
pd.DataFrame(tuned_metrics["Building 1"]).T.sort_values("f1", ascending=False)

### Building 2

In [ ]:
pd.DataFrame(tuned_metrics["Building 2"]).T.sort_values("f1", ascending=False)

### Building 3

In [ ]:
pd.DataFrame(tuned_metrics["Building 3"]).T.sort_values("f1", ascending=False)